# Management Workflow 테스트

이 노트북은 management 워크플로우를 직접 실행하여 테스트합니다.

In [ ]:
# 환경 변수 로드
import os
import sys
from dotenv import load_dotenv

# 프로젝트 루트 디렉토리를 Python 경로에 추가
project_root = os.path.abspath('.')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# .env 파일 로드
load_dotenv()

# 환경 변수 확인
print(f"OPENAI_API_KEY: {os.getenv('OPENAI_API_KEY')[:20]}...")
print(f"TAVILY_API_KEY: {os.getenv('TAVILY_API_KEY')[:20]}...")

## 1. 필요한 모듈 임포트

In [ ]:
from agents.management.modules.state import ManagementState
from agents.management.modules.nodes import WebSearchNode, ResourceManagementNode
from agents.management.workflow import ManagementWorkflow

## 2. 개별 노드 테스트

In [ ]:
# 웹 검색 노드 테스트
web_search_node = WebSearchNode()

# 초기 상태 생성
test_state = {
    "project_id": "PRJ-2024-001",
    "request_type": "resource_allocation",
    "query": "콘서트 기획",
    "response": [],
    "team_members": ["기획팀", "마케팅팀", "기술팀"],
    "resources_available": {"budget": "5억원", "venue": "올림픽공원"}
}

# 웹 검색 실행
print("=== 웹 검색 노드 실행 ===")
search_result = web_search_node.execute(test_state)
print(f"검색 결과: {search_result}")

In [ ]:
# 검색 결과를 상태에 추가
test_state["search_results"] = search_result.get("search_results", "검색 결과 없음")

# 리소스 관리 노드 테스트
resource_node = ResourceManagementNode()

print("\n=== 리소스 관리 노드 실행 ===")
resource_result = resource_node.execute(test_state)
print(f"\n리소스 계획:\n{resource_result['response']}")

## 3. 전체 워크플로우 실행

In [ ]:
# ManagementWorkflow 인스턴스 생성
workflow = ManagementWorkflow()

# 워크플로우 컴파일
compiled_workflow = workflow.build()

# 워크플로우 구조 확인
print("워크플로우 그래프 구조:")
print(compiled_workflow.get_graph())

In [ ]:
# 워크플로우 실행을 위한 입력 데이터
workflow_input = {
    "project_id": "PRJ-2024-002",
    "request_type": "team_management",
    "query": "신규 드라마 제작팀 구성",
    "response": [],
    "team_members": ["PD", "작가", "촬영감독"],
    "resources_available": {
        "budget": "100억원",
        "duration": "6개월",
        "location": "서울 및 부산"
    }
}

print("\n=== 전체 워크플로우 실행 ===")
print(f"입력: {workflow_input}\n")

# 워크플로우 실행
result = compiled_workflow.invoke(workflow_input)

print("\n=== 최종 결과 ===")
print(f"프로젝트 ID: {result['project_id']}")
print(f"요청 유형: {result['request_type']}")
print(f"\n리소스 계획:\n{result.get('response', '응답 없음')}")

## 4. 다양한 시나리오 테스트

In [ ]:
# 시나리오 1: 음악 앨범 제작
music_input = {
    "project_id": "MUSIC-2024-IU",
    "request_type": "resource_allocation",
    "query": "새 앨범 제작을 위한 리소스 계획",
    "response": [],
    "team_members": ["프로듀서", "작곡가", "편곡자", "엔지니어"],
    "resources_available": {
        "budget": "20억원",
        "studio": "서울 스튜디오 A, B",
        "timeline": "3개월"
    }
}

print("=== 시나리오 1: 음악 앨범 제작 ===")
music_result = compiled_workflow.invoke(music_input)
print(f"\n리소스 계획:\n{music_result.get('response', '응답 없음')}")

In [ ]:
# 시나리오 2: 팬미팅 이벤트
fanmeeting_input = {
    "project_id": "EVENT-2024-FM",
    "request_type": "creator_development",
    "query": "글로벌 팬미팅 투어 계획",
    "response": [],
    "team_members": ["이벤트 기획팀", "보안팀", "통역팀"],
    "resources_available": {
        "budget": "30억원",
        "cities": "서울, 도쿄, 방콕, 홍콩",
        "duration": "2주"
    }
}

print("\n=== 시나리오 2: 팬미팅 이벤트 ===")
fanmeeting_result = compiled_workflow.invoke(fanmeeting_input)
print(f"\n리소스 계획:\n{fanmeeting_result.get('response', '응답 없음')}")

## 5. 스트리밍 모드로 실행 (실시간 진행 상황 확인)

In [ ]:
# 스트리밍 모드로 워크플로우 실행
streaming_input = {
    "project_id": "STREAM-2024-001",
    "request_type": "resource_allocation",
    "query": "넷플릭스 시리즈 제작",
    "response": [],
    "team_members": ["총괄 PD", "각본가", "캐스팅 디렉터"],
    "resources_available": {
        "budget": "200억원",
        "episodes": "8부작",
        "platform": "Netflix"
    }
}

print("=== 스트리밍 모드 실행 ===")
for event in compiled_workflow.stream(streaming_input):
    for node_name, node_output in event.items():
        print(f"\n[{node_name}] 실행 중...")
        if isinstance(node_output, dict):
            for key, value in node_output.items():
                if key == "response" and value:
                    print(f"응답: {value[:200]}...")
                elif key == "search_results":
                    print(f"검색 완료: {value[:100]}...")

## 6. 커스텀 도구 테스트 (Tavily 없이)

In [ ]:
# Tavily API 키 임시 제거하여 모의 검색 도구 사용
original_tavily_key = os.environ.get('TAVILY_API_KEY')
if 'TAVILY_API_KEY' in os.environ:
    del os.environ['TAVILY_API_KEY']

# 모의 검색 도구로 워크플로우 재생성
mock_workflow = ManagementWorkflow()
mock_compiled = mock_workflow.build()

mock_input = {
    "project_id": "MOCK-2024-001",
    "request_type": "resource_allocation",
    "query": "모의 프로젝트 테스트",
    "response": [],
    "team_members": ["테스트팀"],
    "resources_available": {"budget": "테스트"}
}

print("=== 모의 검색 도구 테스트 ===")
mock_result = mock_compiled.invoke(mock_input)
print(f"\n결과: {mock_result.get('response', '응답 없음')[:500]}...")

# 원래 키 복원
if original_tavily_key:
    os.environ['TAVILY_API_KEY'] = original_tavily_key